# Computations of emissions from retail customers
* Includes only emissins related to mortgage loans and not deposits

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Global variables
ABS_FILE_PATH = "..."

## An energy grading system to categorize the energy consumption of all buildnings
Assuming energy grade _G_ takes the same values as grade _F_

_Arealkorreksjon_ is an additional number that is used in computing net energy usage for småhus and leiligheter. Let _grade_building_ be the kWh used per square meter. Then, the formula is:
* net_energy_usage = area_sqmt * (grade_building + Arealkorreksjon / area_sqmt)

In [ ]:
url_energyGrades = "https://www.energimerking.no/download?objectPath=/upload_images%2FFC1EA4B47E3141AFAD2CFD75C7829E6F.xlsx"

df_energyGrade = pd.read_excel(url_energyGrades, skiprows=4, skipfooter=43)
df_energyGrade = df_energyGrade.drop(0).rename(columns={'Unnamed: 0': "Type"}).set_index("Type", drop=True)

# not at all correct but works for now
df_energyGrade["G"] = df_energyGrade["F"]

# Somehow the second 'Arealkorreksjon' is noted '   Arealkorreksjon' in the DataFrame
df_energyGrade.loc["Arealkorreksjon"] = df_energyGrade.loc["Arealkorreksjon"].str.extract('(\d+)', expand=False).astype(int)
df_energyGrade.loc['   Arealkorreksjon'] = df_energyGrade.loc['   Arealkorreksjon'].str.extract('(\d+)', expand=False).astype(int)
df_energyGrade

In [ ]:
df_energyGrade_smallhouse = pd.read_excel(url_energyGrades, sheet_name="E-skala v7.24 Smaahus",skiprows=4, skipfooter=3, usecols=["Unnamed: 0", "A","B","C","D","E","F","G"])
df_energyGrade_smallhouse = df_energyGrade_smallhouse.rename(columns={"Unnamed: 0":"Størrelse"}).set_index("Størrelse", drop=True).drop("Oppvarmet\nBRA (m2)")
df_energyGrade_smallhouse

In [ ]:
df_energyGrade_apartment = pd.read_excel(url_energyGrades, sheet_name="E-skala v7.24 Leilighet",skiprows=4, skipfooter=3, usecols=["Unnamed: 0", "A","B","C","D","E","F","G"])
df_energyGrade_apartment = df_energyGrade_apartment.rename(columns={"Unnamed: 0":"Størrelse"}).set_index("Størrelse", drop=True).drop("Oppvarmet\nBRA (m2)")
df_energyGrade_apartment

## Group BNB portfolio into energy grades

In [ ]:
# internal file 
file = ABS_FILE_PATH + '....xlsx'

df_eiendomsverdi = pd.read_excel(file)

lst_BNB_dist_energygrades = [df_eiendomsverdi[df_eiendomsverdi['Energikarakter'] == grade]['Energikarakter'].count() for grade in ['A', 'B', 'C', 'D', 'E', 'F', 'G']]
num_total = sum(lst_BNB_dist_energygrades)

# Compute median LTV per energy grade. Does not seem to be any specific correlation between energy grade and LTV.
lst_energygrade_ltv = [df_eiendomsverdi[df_eiendomsverdi["Energikarakter"] == grade]["LTV_INTERN"].median() for grade in ['A', 'B', 'C', 'D', 'E', 'F', 'G']] 

df_BNB_pm_energygrades = pd.DataFrame(index=df_energyGrade.columns)
df_BNB_pm_energygrades["Share[%]"] = lst_BNB_dist_energygrades / num_total * 100
df_BNB_pm_energygrades["LTV"] = lst_energygrade_ltv

df_BNB_pm_energygrades

In [ ]:
file = ABS_FILE_PATH + '....xlsx'
df_pm_portfolio = pd.read_excel(file)

df_pm_portfolio = df_pm_portfolio[['NIVAA1_KUNDE','BOLIG_KVM','ASSET_DESC']].rename(columns={'NIVAA1_KUNDE':'Type','BOLIG_KVM':'Størrelse','ASSET_DESC':'Kategori'})
df_pm_portfolio = df_pm_portfolio[df_pm_portfolio['Type']=='PM']

df_pm_portfolio

In [ ]:
PM_total_units = df_pm_portfolio["Type"].count()
PM_total_units

df_BNB_pm_energygrades['Num_units'] = [int(round(df_BNB_pm_energygrades["Share[%]"].at[grade]/100 * PM_total_units)) for grade in ['A', 'B', 'C', 'D', 'E', 'F', 'G']]
df_BNB_pm_energygrades

### Group portfolio into small houses and apartments

In [ ]:
small_houses = ['Enebolig', 'Rekkehus', 'Boligeiendom', 'Fritidseiendom', 'Hytte/fritidseiendom', 'Småbruk',
              'Enebolig i kjede', 'Bolig og forr.bygg', 'Forretningsbygg', 'Næringseiendom','Gårdsbruk',  
               'Næringseiendom (leiekontrakt)', 'Næringseiendom m/boligdel', 'Landbrukseiendom m/bolig']

apartments = ['Selveierleilighet', 'Leilighet', 'Andel i borettslag', 'Delt bolig', 
              'Tomt', 'Terrasseleilighet', 'Ubebygget tomt', 'Borettslag']

total_small_house = df_pm_portfolio[df_pm_portfolio["Kategori"].isin(small_houses)]["Type"].count()
total_apartments = df_pm_portfolio[df_pm_portfolio["Kategori"].isin(apartments)]["Type"].count()


### Distributing houses and apartments on energy grades

In [ ]:
df_BNB_pm_energygrades['Small_house'] = [int(round(df_BNB_pm_energygrades["Share[%]"][grade]/100 * total_small_house)) for grade in ['A', 'B', 'C', 'D', 'E', 'F', 'G']]
df_BNB_pm_energygrades['Apartment'] = [int(round(df_BNB_pm_energygrades["Share[%]"][grade]/100 * total_apartments)) for grade in ['A', 'B', 'C', 'D', 'E', 'F', 'G']]

df_BNB_pm_energygrades

#### Finding median sizes for small houses and apartments

In [ ]:
s_apartment_size = df_pm_portfolio[df_pm_portfolio["Kategori"].isin(apartments)]["Størrelse"]
s_apartment_size = pd.to_numeric(s_apartment_size, errors='coerce')
median_apartments = s_apartment_size.median()

s_small_houses_size = df_pm_portfolio[df_pm_portfolio["Kategori"].isin(small_houses)]["Størrelse"]
s_small_houses_size = pd.to_numeric(s_small_houses_size, errors='coerce')
median_small_houses = s_small_houses_size.median()

(median_apartments, median_small_houses)

### Computing net energy usage
Multiplying by the LTV gives yields the part of the building that is part of our portfolio.

In [ ]:
df_BNB_pm_energygrades['Small_house_GWh'] = [df_BNB_pm_energygrades.loc[grade, "Small_house"] * (df_energyGrade.loc["Småhus", grade] + df_energyGrade.loc["Arealkorreksjon", grade] / median_small_houses) * median_small_houses  / 1000000 for grade in ['A', 'B', 'C', 'D', 'E', 'F', 'G']]
df_BNB_pm_energygrades['Apartment_GWh'] = [df_BNB_pm_energygrades.loc[grade, "Apartment"] * (df_energyGrade.loc["Leiligheter (boligblokk)", grade] + df_energyGrade.loc["   Arealkorreksjon", grade] / median_apartments) * median_apartments / 1000000  for grade in ['A', 'B', 'C', 'D', 'E', 'F', 'G']]

# Gets rid of scientifc notation, ie. 3.14e+42
df_BNB_pm_energygrades['Small_house_GWh'] = round(df_BNB_pm_energygrades['Small_house_GWh'] * df_BNB_pm_energygrades['LTV'], 2)
df_BNB_pm_energygrades['Apartment_GWh'] = round(df_BNB_pm_energygrades['Apartment_GWh'] * df_BNB_pm_energygrades['LTV'], 2)

df_BNB_pm_energygrades.at["Total"] = df_BNB_pm_energygrades.sum()
#Summing LTV values makes no sense
df_BNB_pm_energygrades.loc["Total", "LTV"] = 0

In [ ]:
print("Net energy consumption for the private market")
print("df_BNB_pm_energygrades: ")
display(df_BNB_pm_energygrades)